## Imports

In [1]:
using MLDatasets
using Random
import Pkg
Pkg.add("Flux")
using Flux: onehotbatch
Pkg.add("PoissonRandom")
using PoissonRandom
Pkg.add("Distributions")
using Distributions
using Statistics
using Printf

  Updating registry at `~/.julia/registries/General`
 Resolving package versions...


┌ Warning: Some registries failed to update:
│     — `~/.julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Pkg/src/Types.jl:1199


  Updating `/scratch/users/sozcelik19/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `/scratch/users/sozcelik19/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/scratch/users/sozcelik19/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `/scratch/users/sozcelik19/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/scratch/users/sozcelik19/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `/scratch/users/sozcelik19/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [2]:
n_full_test  = 10000 # number of examples to use for full tests  (every epoch)
n_quick_test = 100   # number of examples to use for quick tests (every 1000 examples)

100

## Global Variables

In [3]:
# global variables
l_f_phase      = 2  # length of forward phase (time steps)
l_t_phase      = 2  # length of target phase (time steps)
l_f_phase_test = 2  # length of forward phase for tests (time steps)
dt          = 1.0    # time step (ms)
lambda_max  = 0.2*dt # maximum spike rate (spikes per time step)
# integration_time = 1 # time steps of integration of neuronal variables used for plasticity
# we don't use integration time, this makes index start from 0 where julia indexes start from 1 - directly use 1 inst 0

0.2

In [4]:
# kernel parameters
tau_L = 10.0 # leak time constant

# conductance parameters
g_B = 0.6                                   # basal conductance
g_L = 1.0/tau_L                             # leak conductance
g_D = g_B                                   # dendritic conductance in output layer

E_E = 8                                     # excitation reversal potential
E_I = -8                                    # inhibition reversal potential

# steady state constants
k_B = g_B/(g_L + g_B)
k_D = g_D/(g_L + g_D)
k_I = 1.0/(g_L + g_D)

# weight update constants
P_hidden = 20.0/lambda_max      # hidden layer error signal scaling factor
P_final  = 20.0/(lambda_max^2)  # final layer error signal scaling factor

499.9999999999999

## Functions

In [5]:
# sigmoid function
function sigma(x) 
    return 1 / (1 + exp(-x))
end

sigma (generic function with 1 method)

In [6]:
# derivative sigmoid function
function deriv_sigma(x)
    return exp(-x) / (1 + exp(-x))^2
end

deriv_sigma (generic function with 1 method)

## Objects

In [7]:
mutable struct HiddenLayer
    m # The layer number, eg. m = 1 for the first hidden layer.
    PSP_B # post-synaptic potentials
    B
    A
    C
    lambda_C
    alpha_f
    alpha_t
    E
    delta_W
    delta_b
end

In [8]:
mutable struct FinalLayer
    m # The layer number, ie. m = M where M is the total number of layers.
    lambda_C # something about soma
    PSP_B # post-synaptic potentials
    B
    C
    C_f
    k_D2
    k_E
    k_I
    E
    delta_W
    delta_b
end

In [9]:
mutable struct Network
    n # Tuple - Number of units in each layer of the network, eg. (500, 10) here.
    hiddenLayer::HiddenLayer
    finalLayer::FinalLayer
    W
    b
    Y
    f_etas
    loss
end

## Initialization Sub-Functions

In [10]:
function initializeWbY(ln, n, n_in)
    W = Array{Any}(undef, ln)
    b = Array{Any}(undef, ln)
    Y = Array{Any}(undef, ln-1)
    
    # weight optimization parameters
    V_avg = 3   # desired average of dendritic potential
    V_sd  = 3   # desired standard deviation of dendritic potential
    b_avg = 0.8  # desired average of bias
    b_sd  = 0.001 # desired standard deviation of bias
    nu    = lambda_max*0.25  # slope of linear region of activation function
    V_sm  = V_sd^2 + V_avg^2 # second moment of dendritic potential
    
    for m in 2:-1:1
        # get number of units in the layer below
        if m != 1
            N = n[m-1] #500 W2
        else
            N = n_in  #784 W1
        end
            
        # generate feedforward weights & biases
        # calculate weight variables needed to get desired average & standard deviations of somatic potentials
        W_avg = (V_avg - b_avg)/(nu*N*V_avg) 
        W_sm  = (V_sm + (nu^2)*(N - N^2)*(W_avg^2)*(V_avg^2) - 2*N*nu*b_avg*V_avg*W_avg - (b_avg^2))/(N*(nu^2)*V_sm)
        W_sd  = sqrt(W_sm - W_avg^2)
 
        W[m] = W_avg .+ 3.465*W_sd*rand(Uniform(-1,1), n[m], N) # (500,784) , (10,500)
        b[m] = b_avg .+ 3.465*b_sd*rand(Uniform(-1,1), n[m], 1) # (500,1) , (10,1)
        
        if m != 1
            Y[m-1] = W_avg .+ 3.465*W_sd*rand(Uniform(-1,1), N, n[end]) # (500,10)
        end
    end
        
    return W, b, Y
end

initializeWbY (generic function with 1 method)

## Initializations

In [11]:
# f_input_size - The size of feedforward input, eg. 784 for MNIST input. (784, 1) here.
# b_input_size - The size of feedback input. This is the same as the number of units in the next layer. (10,1) here.
function HiddenLayer(m, n, f_input_size, b_input_size, size_W) # m=1
    PSP_B = zeros(f_input_size, 1)              # (784,1) m=1 in hidden here
    B = zeros(n[m], 1)                          # (500,1) m=1 in hidden here
    A = zeros(n[m], 1)
    C = zeros(n[m], 1)
    lambda_C = zeros(n[m], 1)
    alpha_f = zeros(n[m], 1)
    alpha_t = zeros(n[m], 1)
    E = zeros(n[m], 1)
    delta_W = zeros(size_W)
    delta_b = zeros(n[m], 1)
    return HiddenLayer(m, PSP_B, B, A, C, lambda_C, alpha_f, alpha_t, E, delta_W, delta_b)
end

HiddenLayer

In [12]:
# f_input_size - The size of feedforward input. This is the same as the number of units in the previous layer. (500,1)
function FinalLayer(m, n, f_input_size, size_W) # m=2
    lambda_C = zeros(n[m], 1)                   # (10,1) m=2 in final here
    PSP_B = zeros(f_input_size, 1)              # (500,1) in size 
    B = zeros(n[m], 1)                                
    C = zeros(n[m], 1)
    C_f = zeros(n[m], 1)
    k_D2 = zeros(n[m], 1)
    k_E = zeros(n[m], 1)
    k_I = zeros(n[m], 1)
    E = zeros(n[m], 1)
    delta_W = zeros(size_W)
    delta_b = zeros(n[m], 1)
    return FinalLayer(m, lambda_C, PSP_B, B, C, C_f, k_D2, k_E, k_I, E, delta_W, delta_b)
end

FinalLayer

In [13]:
function Network(n, n_in)
    W, b, Y = initializeWbY(length(n), n, n_in)
    hiddenLayer = HiddenLayer(1, n, n_in, n[end], size(W[1]))
    finalLayer = FinalLayer(2, n, n[end-1], size(W[2]))
    f_etas = (0.21, 0.21)       #Learning rates for each layer's feedforward weights
    loss = 0
    return Network(n, hiddenLayer, finalLayer, W, b, Y, f_etas, loss)
end

Network

## Callable Objects Sub-Functions

In [14]:
# Update basal potentials.
# f_input Feedforward input. (784, 1) here
function update_B(hidden::HiddenLayer, net::Network, f_input)
    hidden.PSP_B = f_input
    hidden.B = net.W[hidden.m]*f_input .+ net.b[hidden.m] # (500,784) * (784,1) .+ (500,1) m=1 in hidden here
end

update_B (generic function with 1 method)

In [15]:
# Update apical potentials.
# b_input Feedback input. (10,1) here
function update_A(hidden::HiddenLayer, net::Network, b_input)
    hidden.A = net.Y[hidden.m]*b_input # (500,10) * (10,1) m=1 in hidden here
end

update_A (generic function with 1 method)

In [16]:
# Update somatic potentials & calculate firing rates.
function update_C(hidden::HiddenLayer, net::Network)
    hidden.C = k_B*hidden.B
    hidden.lambda_C = lambda_max*sigma.(hidden.C)
end

update_C (generic function with 1 method)

In [17]:
# Update basal potentials.
# f_input Feedforward input. (500, 1) here
function update_B(final::FinalLayer, net::Network, f_input)
    final.PSP_B = f_input
    final.B = net.W[final.m]*f_input .+ net.b[final.m] # (10,500) * (500, 1) .+ (10,1) m=2 in final here
end

update_B (generic function with 2 methods)

In [18]:
# Update somatic potentials & calculate firing rates.
function update_C_forward(final::FinalLayer, net::Network)
    final.C = k_D.*final.B
    final.lambda_C = lambda_max*sigma.(final.C)
end

update_C_forward (generic function with 1 method)

In [19]:
# Update injected perisomatic currents.
# b_input : Target input, eg. if the target label is 8,
# b_input = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]).
function update_I(final::FinalLayer, b_input)
    g_E = b_input
    g_I = -g_E .+ 1
    final.k_D2 = g_D./((g_L + g_D) .+ g_E + g_I)
    final.k_E  = g_E./((g_L + g_D) .+ g_E + g_I)
    final.k_I  = g_I./((g_L + g_D) .+ g_E + g_I)
end

update_I (generic function with 1 method)

In [20]:
# Update somatic potentials & calculate firing rates.
function update_C_target(final::FinalLayer, net::Network)
    final.C = final.k_D2.*final.B + final.k_E*E_E + final.k_I*E_I
    final.lambda_C = lambda_max*sigma.(final.C)
end

update_C_target (generic function with 1 method)

## Callable Objects

In [21]:
# hidden layer out_f / out_t
# Perform a forward phase pass. / Perform a target phase pass.
# f_input Feedforward input. (784, 1) here
# b_input Feedback input. (10,1) here

function (hidden::HiddenLayer)(net::Network, f_input, b_input)
    update_B(hidden, net, f_input)
    update_A(hidden, net, b_input)
    update_C(hidden, net)
end

In [22]:
# hidden layer plateau_f / plateau_t
# Calculate forward phase apical plateau potentials.

function (hidden::HiddenLayer)(t_f)
    if t_f == "f"
        # calculate apical calcium spike nonlinearity
        hidden.alpha_f = sigma.(hidden.A)
    end
    if t_f == "t"
        # calculate apical calcium spike nonlinearity
        hidden.alpha_t = sigma.(hidden.A)
    end
end

In [23]:
# hidden layer update_W
# Update feedforward weights.

function (hidden::HiddenLayer)(net::Network)
    hidden.E = (hidden.alpha_t - hidden.alpha_f).*((-k_B)*lambda_max*deriv_sigma.(hidden.C))
    hidden.delta_W = hidden.E * hidden.PSP_B'   #((500,1)*(1,784) = (500,784)
    net.W[hidden.m] += -net.f_etas[hidden.m]*P_hidden*hidden.delta_W
    hidden.delta_b = hidden.E
    net.b[hidden.m] += -net.f_etas[hidden.m]*P_hidden*hidden.delta_b
end

In [24]:
# final layer out_f
# Perform a forward phase pass.
# f_input Feedforward input. (500, 1) here

function (final::FinalLayer)(net::Network, f_input)
    update_B(final, net, f_input)
    update_C_forward(final, net)
end

In [25]:
# final layer out_t
# Perform a target phase pass.
# f_input (ndarray) : Feedforward input.
# b_input (ndarray) : Target input.

function (final::FinalLayer)(net::Network, f_input, b_input)
    update_B(final, net, f_input)
    update_I(final, b_input)
    update_C_target(final, net)
end

In [26]:
# final layer update_W
# Update feedforward weights.

function (final::FinalLayer)(net::Network)
    final.E = (final.lambda_C - lambda_max*sigma.(final.C_f)).*((-k_D)*lambda_max*deriv_sigma.(final.C_f))
    final.delta_W = final.E * final.PSP_B'   #(10,1)*(1,500) = (10,500)
    net.W[final.m] += -net.f_etas[final.m]*P_final*final.delta_W
    final.delta_b = final.E
    net.b[final.m] += -net.f_etas[final.m]*P_final*final.delta_b
end

In [27]:
# net f_phase
# Perform a forward phase.
# x : Input array of size (X, 1) where X is the size of the input, eg. (784, 1).
function (net::Network)(x)
    for time in 1:l_f_phase
        # do a forward pass <--> net out_f -- line 530
        net.hiddenLayer(net, x, net.finalLayer.lambda_C) # hidden layer out_f
        net.finalLayer(net, net.hiddenLayer.lambda_C) # final layer out_f
    end
    # calculate plateau potentials for hidden layer neurons
    net.hiddenLayer("f") #hidden layer plateau_f
    # record C(soma) forward potential
    net.finalLayer.C_f = net.finalLayer.C
end

In [28]:
# net t_phase
# Perform a target phase.
# x : Input array of size (X, 1) where X is the size of the input, eg. (784, 1).
# t : Target array of size (T, 1) where T is the size of the target, eg. (10, 1).

function (net::Network)(x, t)
    for time in 1:l_t_phase
        # do a target pass <--> net out_t
        net.hiddenLayer(net, x, net.finalLayer.lambda_C) # hidden layer out_t
        net.finalLayer(net, net.hiddenLayer.lambda_C, t) # final layer out_t
    end
    # calculate plateau potentials for hidden layer neurons
    net.hiddenLayer("t") #hidden layer plateau_t
    # update weights 
    net.finalLayer(net)       #final layer update_W
    net.hiddenLayer(net)      #hidden layer update_W
    net.loss = mean((net.finalLayer.lambda_C - lambda_max*sigma.(net.finalLayer.C_f)).^2)
    # reset averages
    net.finalLayer.C_f .*= 0
end

## Training Sub-Functions

In [29]:
function clear_vars(net::Network)
    net.hiddenLayer.A .*= 0
    net.hiddenLayer.B .*= 0
    net.hiddenLayer.C .*= 0
    net.hiddenLayer.lambda_C .*= 0

    net.hiddenLayer.PSP_B .*= 0

    net.hiddenLayer.E .*= 0
    net.hiddenLayer.delta_W .*= 0
    net.hiddenLayer.delta_b .*= 0

    net.hiddenLayer.alpha_f .*= 0
    net.hiddenLayer.alpha_t .*= 0
    
    net.finalLayer.B .*= 0
    net.finalLayer.C .*= 0
    net.finalLayer.lambda_C .*= 0

    net.finalLayer.PSP_B .*= 0

    net.finalLayer.E .*= 0
    net.finalLayer.delta_W .*= 0
    net.finalLayer.delta_b .*= 0

    net.finalLayer.C_f .*= 0
end

clear_vars (generic function with 1 method)

In [30]:
# net test_weights
# Test the network's current weights on the test set. The network's layers are copied
#      and restored to their previous state after testing.
# n_test : The number of test examples to use.

function test_weights(net::Network, n_test, x_test, t_test, l_f_phase, l_f_phase_test)
    # save old length of forward phase
    old_l_f_phase = l_f_phase

    # set new length of forward phase
    l_f_phase = l_f_phase_test

    # initialize count of correct classifications
    num_correct = 0

    # shuffle testing data
    order = randperm(size(x_test)[2]) #randperm(10000)
    x_test = x_test[:,order] 
    t_test = t_test[:,order]
    
    for i in 1:n_test
        # clear all layer variables
        clear_vars(net)

        # get testing example data
        x = lambda_max*x_test[:, i]
        t = t_test[:, i]

        # do a forward phase & get the unit with maximum average somatic potential
        net(x) # net f_phase
        sel_num = argmax(net.finalLayer.C_f)[1]

        # get the target number from testing example data
        target_num = argmax(t)

        # increment correct classification counter if they match
        if sel_num == target_num
            num_correct += 1
        end
    end
    
    # calculate percent error
    err_rate = (1.0 - num_correct/n_test)*100.0

    l_f_phase = old_l_f_phase

    clear_vars(net)
    
    return err_rate
end

test_weights (generic function with 1 method)

## Training

In [31]:
function load_MNIST()
    x_train = MNIST.convert2features(MNIST.traintensor(Float64))
    t_train = MNIST.trainlabels()
    t_train = onehotbatch(t_train, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])    
    x_test = MNIST.convert2features(MNIST.testtensor(Float64))
    t_test = MNIST.testlabels()
    t_test = onehotbatch(t_test, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) 
    return x_train, t_train, x_test, t_test
end

load_MNIST (generic function with 1 method)

In [32]:
# net train
function train(net::Network, n_epochs = 10, n_training_examples = 60000)
    
    x_train, t_train, x_test, t_test = load_MNIST()
    
    current_epoch = 1
    
    #record_training_labels:
    training_labels = zeros(n_epochs*n_training_examples)
    
    # don't do an initial weight test
    println("Start of epoch $(current_epoch)")
    
    # start time used for timing how long each 1000 examples take
    start_time = nothing
    
    #record_training_error
    num_correct = 0
    
    for k in 1:n_epochs
        # shuffle the training data
        order = randperm(size(x_train)[2]) #randperm(60000)
        x_train = x_train[:,order] 
        t_train = t_train[:,order]
                  
        for i in 1:n_training_examples   # n become i here
            # set start time
            if start_time == nothing
                start_time = time()
            end
            
            # get training example data
            x = lambda_max*x_train[:,i]
            t = t_train[:,i]
            
            # do forward phase
            net(x) # net f_phase
            
            sel_num = argmax(net.finalLayer.C_f)[1] #axis=-1

            # get the target number from testing example data
            target_num = argmax(t)

            # increment correct classification counter if they match
            if sel_num == target_num
                num_correct += 1
            end
            
            # do target phase
            net(x, t) # net t_phase
            
            #record_training_labels
            training_labels[(k-1)*n_training_examples + i] = argmax(t)
            
            if i % 1000 == 0
                if i != n_training_examples
                    # we're partway through an epoch; do a quick weight test
                    test_err = test_weights(net, n_quick_test, x_test, t_test, l_f_phase, l_f_phase_test)
                    @printf("Epoch %d, example %d/%d. QE: %g ", current_epoch, i, n_training_examples, test_err)
                else
                    # we've reached the end of an epoch; do a full weight test
                    test_err = test_weights(net, n_full_test, x_test, t_test, l_f_phase, l_f_phase_test)
                    @printf("FE: %g ", test_err)
                    
                    # calculate percent training error for this epoch
                    err_rate = (1.0 - num_correct/n_training_examples)*100.0
                    @printf("TE: %g ", err_rate)

                    num_correct = 0
                end
                # get end time & reset start time
                end_time = time()
                time_elapsed = end_time - start_time
                @printf("T: %g\n", time_elapsed)
                start_time = nothing
            end
        end
        # update latest epoch counter
        current_epoch += 1
    end
end

train (generic function with 3 methods)

In [33]:
n = (500, 10) #for now building everything for having one hidden layer
n_in  = 784
net = Network(n, n_in)
train(net)

Start of epoch 1
Epoch 1, example 1000/60000. QE: 57 T: 7.83941
Epoch 1, example 2000/60000. QE: 27 T: 2.58181
Epoch 1, example 3000/60000. QE: 29 T: 2.52751
Epoch 1, example 4000/60000. QE: 11 T: 2.5638
Epoch 1, example 5000/60000. QE: 10 T: 2.51279
Epoch 1, example 6000/60000. QE: 16 T: 2.52604
Epoch 1, example 7000/60000. QE: 17 T: 2.52399
Epoch 1, example 8000/60000. QE: 9 T: 2.53095
Epoch 1, example 9000/60000. QE: 13 T: 2.49399
Epoch 1, example 10000/60000. QE: 11 T: 2.50501
Epoch 1, example 11000/60000. QE: 10 T: 2.5263
Epoch 1, example 12000/60000. QE: 10 T: 2.51669
Epoch 1, example 13000/60000. QE: 23 T: 2.50286
Epoch 1, example 14000/60000. QE: 12 T: 2.50643
Epoch 1, example 15000/60000. QE: 7 T: 2.50977
Epoch 1, example 16000/60000. QE: 13 T: 2.52284
Epoch 1, example 17000/60000. QE: 15 T: 2.56705
Epoch 1, example 18000/60000. QE: 4 T: 2.49976
Epoch 1, example 19000/60000. QE: 10 T: 2.50876
Epoch 1, example 20000/60000. QE: 13 T: 2.51305
Epoch 1, example 21000/60000. QE: 5 T

Epoch 3, example 56000/60000. QE: 2 T: 2.46253
Epoch 3, example 57000/60000. QE: 3 T: 2.48152
Epoch 3, example 58000/60000. QE: 8 T: 2.46586
Epoch 3, example 59000/60000. QE: 2 T: 2.49767
FE: 3.65 TE: 4.29833 T: 7.93656
Epoch 4, example 1000/60000. QE: 3 T: 2.69868
Epoch 4, example 2000/60000. QE: 5 T: 2.72969
Epoch 4, example 3000/60000. QE: 6 T: 2.80782
Epoch 4, example 4000/60000. QE: 3 T: 2.73019
Epoch 4, example 5000/60000. QE: 4 T: 2.65893
Epoch 4, example 6000/60000. QE: 7 T: 2.75943
Epoch 4, example 7000/60000. QE: 5 T: 2.76635
Epoch 4, example 8000/60000. QE: 5 T: 2.73709
Epoch 4, example 9000/60000. QE: 2 T: 2.76585
Epoch 4, example 10000/60000. QE: 5 T: 2.75935
Epoch 4, example 11000/60000. QE: 4 T: 2.77221
Epoch 4, example 12000/60000. QE: 6 T: 2.73631
Epoch 4, example 13000/60000. QE: 3 T: 2.76136
Epoch 4, example 14000/60000. QE: 3 T: 2.76479
Epoch 4, example 15000/60000. QE: 4 T: 2.83994
Epoch 4, example 16000/60000. QE: 5 T: 2.72441
Epoch 4, example 17000/60000. QE: 1 T

Epoch 6, example 53000/60000. QE: 3 T: 2.77733
Epoch 6, example 54000/60000. QE: 3 T: 2.72195
Epoch 6, example 55000/60000. QE: 5 T: 2.8037
Epoch 6, example 56000/60000. QE: 3 T: 2.72286
Epoch 6, example 57000/60000. QE: 5 T: 2.7847
Epoch 6, example 58000/60000. QE: 5 T: 2.71965
Epoch 6, example 59000/60000. QE: 3 T: 2.79387
FE: 2.93 TE: 2.54167 T: 8.29718
Epoch 7, example 1000/60000. QE: 3 T: 3.07824
Epoch 7, example 2000/60000. QE: 4 T: 2.45793
Epoch 7, example 3000/60000. QE: 2 T: 2.44816
Epoch 7, example 4000/60000. QE: 3 T: 2.47574
Epoch 7, example 5000/60000. QE: 5 T: 2.46831
Epoch 7, example 6000/60000. QE: 3 T: 2.48092
Epoch 7, example 7000/60000. QE: 2 T: 2.46565
Epoch 7, example 8000/60000. QE: 4 T: 2.48813
Epoch 7, example 9000/60000. QE: 2 T: 2.44898
Epoch 7, example 10000/60000. QE: 4 T: 2.52824
Epoch 7, example 11000/60000. QE: 4 T: 2.45705
Epoch 7, example 12000/60000. QE: 5 T: 2.49495
Epoch 7, example 13000/60000. QE: 6 T: 2.45653
Epoch 7, example 14000/60000. QE: 4 T: 

Epoch 9, example 50000/60000. QE: 1 T: 2.45398
Epoch 9, example 51000/60000. QE: 3 T: 2.48095
Epoch 9, example 52000/60000. QE: 2 T: 2.46358
Epoch 9, example 53000/60000. QE: 4 T: 2.48994
Epoch 9, example 54000/60000. QE: 1 T: 2.4657
Epoch 9, example 55000/60000. QE: 3 T: 2.47891
Epoch 9, example 56000/60000. QE: 1 T: 2.45195
Epoch 9, example 57000/60000. QE: 1 T: 2.45822
Epoch 9, example 58000/60000. QE: 7 T: 2.46065
Epoch 9, example 59000/60000. QE: 4 T: 2.50089
FE: 3.28 TE: 1.81833 T: 7.84865
Epoch 10, example 1000/60000. QE: 3 T: 2.734
Epoch 10, example 2000/60000. QE: 4 T: 2.73159
Epoch 10, example 3000/60000. QE: 2 T: 2.77228
Epoch 10, example 4000/60000. QE: 1 T: 2.74972
Epoch 10, example 5000/60000. QE: 5 T: 2.63265
Epoch 10, example 6000/60000. QE: 6 T: 2.72299
Epoch 10, example 7000/60000. QE: 2 T: 2.75217
Epoch 10, example 8000/60000. QE: 0 T: 2.74664
Epoch 10, example 9000/60000. QE: 0 T: 2.82487
Epoch 10, example 10000/60000. QE: 3 T: 2.72527
Epoch 10, example 11000/60000.